In [18]:
import itertools
import torch
import torch.optim as optim
from tensorflow import keras
from PIL import Image
from torchvision import models, transforms
import torch.nn as nn
import random
from numpy.linalg import norm
import numpy as np
import math
import time
from torch.utils.data import DataLoader, TensorDataset
from sklearn.decomposition import PCA

In [19]:
import sys
root = '../../../'
sys.path.append(root)
from HelpfulFunctions.batchCreation import createBatch
from HelpfulFunctions.metrics import meanAveragePrecision
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
def CreateDataset(root, num_classes, batch_size, train = 1, HPO = False):
    if HPO == False:
        if train == 1:
            #Create X_train_tensor
            X_train = np.load( root + "Features/train_features_vgg16_cifar10.npy" ) # Shape = (45000, 4096)

            #pca = PCA(n_components=128)
            #pca.fit(X_train)
            #X_train = torch.tensor(pca.transform(X_train), dtype=torch.float)
            #X_test = torch.tensor(pca.transform(X_test), dtype=torch.float)
            X_train_tensor = torch.tensor(X_train)

            #Create Y_train_tensor
            y_train = np.load( root + "Features/train_labels_vgg16_cifar10.npy" ) # Shape = (45000,)
            y_train_tensor = torch.tensor(y_train, dtype=torch.long)
            y_train_tensor = torch.nn.functional.one_hot(y_train_tensor, num_classes) #One-Hot Encoded -> Shape = (45000, num_classes)

            #Create indices
            indices_train = torch.arange(len(X_train_tensor))

            dataset = TensorDataset(X_train_tensor, y_train_tensor, indices_train)
            train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            return train_loader

        elif train == 2:
            X_validation = np.load( root + "Features/val_features_vgg16_cifar10.npy" ) # Shape = (10000, 4096)
            X_validation_tensor = torch.tensor(X_validation)

            y_validation = np.load( root + "Features/val_labels_vgg16_cifar10.npy" ) # Shape = (10000,)
            y_validation_tensor = torch.tensor(y_validation, dtype=torch.long)
            y_validation_tensor = torch.nn.functional.one_hot(y_validation_tensor, num_classes) #One-Hot Encoded -> Shape = (10000, num_classes)

            #Create indices
            indices_validation = torch.arange(len(X_validation_tensor))

            dataset = TensorDataset(X_validation_tensor, y_validation_tensor, indices_validation)
            validation_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            return validation_loader

        elif train == 3:
            X_test = np.load( root + "Features/test_features_vgg16_cifar10.npy" ) # Shape = (10000, 4096)
            X_test_tensor = torch.tensor(X_test)

            y_test = np.load( root + "Features/test_labels_vgg16_cifar10.npy" ) # Shape = (10000,)
            y_test_tensor = torch.tensor(y_test, dtype=torch.long)
            y_test_tensor = torch.nn.functional.one_hot(y_test_tensor, num_classes) #One-Hot Encoded -> Shape = (10000, num_classes)

            #Create indices
            indices_test = torch.arange(len(X_test_tensor))

            dataset = TensorDataset(X_test_tensor, y_test_tensor, indices_test)
            test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            return test_loader



    if HPO == True:
        if train == 1:
            #Create X_train_tensor
            X_train = np.load( root + "Features/X_hpo_Cifar.npy" ) # Shape = (45000, 4096)
            X_train_tensor = torch.tensor(X_train)
            #print(X_train_tensor.shape)

            #Create Y_train_tensor
            y_train = np.load( root + "Features/y_hpo_CIfar.npy" ) # Shape = (45000,)
            y_train_tensor = torch.tensor(y_train, dtype=torch.long)
            y_train_tensor = torch.nn.functional.one_hot(y_train_tensor, num_classes) #One-Hot Encoded -> Shape = (45000, num_classes)
            #print(y_train_tensor.shape)

            #Create indices
            indices_train = torch.arange(len(X_train_tensor))

            dataset = TensorDataset(X_train_tensor, y_train_tensor, indices_train)
            train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            return train_loader

        elif train == 2:
            X_validation = np.load( root + "Features/X_val_Cifar.npy" ) # Shape = (10000, 4096)
            X_validation_tensor = torch.tensor(X_validation)
            #print(X_validation_tensor.shape)

            y_validation = np.load( root + "Features/y_val_Cifar.npy" ) # Shape = (10000,)
            y_validation_tensor = torch.tensor(y_validation, dtype=torch.long)
            y_validation_tensor = torch.nn.functional.one_hot(y_validation_tensor, num_classes) #One-Hot Encoded -> Shape = (10000, num_classes)
            #print(y_validation_tensor.shape)

            #Create indices
            indices_validation = torch.arange(len(X_validation_tensor))

            dataset = TensorDataset(X_validation_tensor, y_validation_tensor, indices_validation)
            validation_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            return validation_loader

        elif train == 3:
            X_test = np.load( root + "Features/test_features_vgg16_cifar10.npy") # Shape = (10000, 4096)
            X_test_tensor = torch.tensor(X_test)

            y_test = np.load( root + "Features/test_labels_vgg16_cifar10.npy") # Shape = (10000,)
            y_test_tensor = torch.tensor(y_test, dtype=torch.long)
            y_test_tensor = torch.nn.functional.one_hot(y_test_tensor, num_classes) #One-Hot Encoded -> Shape = (10000, num_classes)

            #Create indices
            indices_test = torch.arange(len(X_test_tensor))

            dataset = TensorDataset(X_test_tensor, y_test_tensor, indices_test)
            test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            return test_loader

In [21]:
train_loader = CreateDataset(root, num_classes = 10, batch_size = 128, train = 1, HPO=True)
#test_loader = CreateDataset(root, num_classes = 10, batch_size = 128, train = 2)
validation_loader = CreateDataset(root, num_classes = 10, batch_size = 128, train = 2, HPO = True)

In [22]:
class CustomNN(nn.Module):
    def __init__(self, bits):
        super(CustomNN, self).__init__()
        self.fc_layers = nn.Sequential(
            nn.Linear(4096, 1024),  # First fully connected layer
            nn.ReLU(),
            nn.Linear(1024, bits),    # Second fully connected layer to reduce to 4000
        )

        # Initialize weights and biases from gaussian distribution
        for layer in self.fc_layers:
            if isinstance(layer, nn.Linear):
                nn.init.normal_(layer.weight, mean=0.0, std=0.01)  # Initialize weights based on paper
                nn.init.normal_(layer.bias, mean=0.0, std=0.01)    # Initialize biases based on paper

    def forward(self, x):
        return self.fc_layers(x)

In [23]:
class DPSHLoss(torch.nn.Module):
    def __init__(self, train_size, n_classes, bit):
        super(DPSHLoss, self).__init__()
        self.U = torch.zeros(train_size, bit).float().to(device)
        self.Y = torch.zeros(train_size, n_classes).float().to(device)

    def forward(self, u, y, ind, eta):
        self.U[ind, :] = u.data
        self.Y[ind, :] = y.float()

        s = (y @ self.Y.t() > 0).float().clamp(max = 1)
        inner_product = u @ self.U.t() * 0.5

        likelihood_loss = (1 + (-(inner_product.abs())).exp()).log() + inner_product.clamp(min=0) - s * inner_product

        likelihood_loss = likelihood_loss.mean()

        quantization_loss = eta * (u - u.sign()).pow(2).mean()

        return likelihood_loss + quantization_loss

In [24]:
# Define the grid

def DPSH(device: torch.device, train_size: int, n_classes: int, bit: int, num_epoch: int, batch_size: int, eta_values: list, wd_values: list, lr_values: list):

    train_loader = CreateDataset(root, num_classes = 10, batch_size = 128, train = 1, HPO=True)
    #test_loader = CreateDataset(root, num_classes = 10, batch_size = 128, train = 2)
    validation_loader = CreateDataset(root, num_classes = 10, batch_size = 128, train = 2, HPO = True)




    param_grid = {
        'eta': eta_values,
        'learning_rate': lr_values,
        #'batch_size': [16, 32, 64],
        'weight_decay': wd_values
    }

    customLoss = DPSHLoss(train_size, n_classes, bit)


    # Get all combinations of parameters
    keys, values = zip(*param_grid.items())
    parameter_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    # Evaluate each parameter combination
    best_params = None
    best_map = 0

    for params in parameter_combinations:
        print(f"Testing combination: {params}")
    
        # Initialize loss function with specific parameters
        #loss_fn = customLoss()

        # Initialize model and optimizer
        model = CustomNN(bits = bit).to(device)
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'], weight_decay = params['weight_decay'])

        # Train the model
        for epoch in range(num_epoch):  # Example epoch count
            current_time = time.strftime('%H:%M:%S', time.localtime(time.time()))
            print("%s[%2d/%2d][%s] bit:%d, dataset:%s, training...." % (
            "DPSH", epoch + 1, num_epoch, current_time, bit, "CIFAR"), end="")
            model.train()
            train_loss = 0
            for image, label, ind in train_loader:
                image = image.to(device)
                label = label.to(device)

                optimizer.zero_grad()
                u = model(image)

                loss = customLoss(u, label.float(), ind, eta = params['eta'])
                train_loss += loss.item()

                loss.backward()
                optimizer.step()

            train_loss = train_loss / (train_size / batch_size)
            print("\b\b\b\b\b\b\b loss:%.5f" % (train_loss))
        # Validate the model
        model.eval()
        hash_train = []
        label_train = []

        hash_val = []
        label_val = []

        with torch.no_grad():
            for image, label, ind in train_loader:
                image = image.to(device)
                label_batch = label.to(device)
                hash_train_batch = (model(image)).sign()
                hash_train.append(hash_train_batch)
                label_train.append(label_batch)
                #hash_codes_matrix = np.vstack(hash_code_batches)
                #print(hash_codes_matrix.shape) 

            hash_train = torch.cat(hash_train, dim = 0).cpu().numpy()
            label_train = torch.cat(label_train, dim = 0).cpu().numpy()

            
            for image, label, ind in validation_loader:
                image = image.to(device)
                label_batch = label.to(device)
                hash_val_batch = (model(image)).sign()
                hash_val.append(hash_val_batch)
                label_val.append(label_batch)
            hash_val = torch.cat(hash_val, dim = 0).cpu().numpy()
            label_val = torch.cat(label_val, dim = 0).cpu().numpy()

            map = meanAveragePrecision(training_hashes = hash_train, training_labels = label_train, test_hashes = hash_val, test_labels = label_val)

            print(f"Validation mAP: {map:.4f}")

            # Update best parameters
            if map > best_map:
                best_map = map
                best_params = params

    print("Best Parameters:", best_params)
    print("Best mAP:", best_map)


In [ ]:
DPSH(device, 5000, 10, 24, 150, 128, [0.01, 0.05, 0.1], [1e-6, 1e-5, 1e-4], [1e-6, 1e-5, 1e-4])

Testing combination: {'eta': 0.01, 'learning_rate': 1e-06, 'weight_decay': 1e-06}
DPSH[ 1/150][10:39:10] bit:24, dataset:CIFAR, training... loss:0.75330
DPSH[ 2/150][10:39:11] bit:24, dataset:CIFAR, training... loss:0.71506
DPSH[ 3/150][10:39:11] bit:24, dataset:CIFAR, training... loss:0.72560
DPSH[ 4/150][10:39:11] bit:24, dataset:CIFAR, training... loss:0.71337
DPSH[ 5/150][10:39:11] bit:24, dataset:CIFAR, training... loss:0.71628
DPSH[ 6/150][10:39:11] bit:24, dataset:CIFAR, training... loss:0.71272
DPSH[ 7/150][10:39:12] bit:24, dataset:CIFAR, training... loss:0.71190
DPSH[ 8/150][10:39:12] bit:24, dataset:CIFAR, training... loss:0.71122
DPSH[ 9/150][10:39:12] bit:24, dataset:CIFAR, training... loss:0.71073
DPSH[10/150][10:39:12] bit:24, dataset:CIFAR, training... loss:0.71258
DPSH[11/150][10:39:12] bit:24, dataset:CIFAR, training... loss:0.71075
DPSH[12/150][10:39:13] bit:24, dataset:CIFAR, training... loss:0.71196
DPSH[13/150][10:39:13] bit:24, dataset:CIFAR, training... loss:0.7

100%|██████████| 500/500 [00:01<00:00, 441.61it/s]


Validation mAP: 0.5282
Testing combination: {'eta': 0.01, 'learning_rate': 1e-06, 'weight_decay': 1e-05}
DPSH[ 1/150][10:39:40] bit:24, dataset:CIFAR, training... loss:0.75187
DPSH[ 2/150][10:39:40] bit:24, dataset:CIFAR, training... loss:0.71573
DPSH[ 3/150][10:39:40] bit:24, dataset:CIFAR, training... loss:0.72565
DPSH[ 4/150][10:39:40] bit:24, dataset:CIFAR, training... loss:0.71405
DPSH[ 5/150][10:39:41] bit:24, dataset:CIFAR, training... loss:0.71624
DPSH[ 6/150][10:39:41] bit:24, dataset:CIFAR, training... loss:0.71246
DPSH[ 7/150][10:39:41] bit:24, dataset:CIFAR, training... loss:0.71111
DPSH[ 8/150][10:39:41] bit:24, dataset:CIFAR, training... loss:0.71095
DPSH[ 9/150][10:39:41] bit:24, dataset:CIFAR, training... loss:0.70865
DPSH[10/150][10:39:42] bit:24, dataset:CIFAR, training... loss:0.70890
DPSH[11/150][10:39:42] bit:24, dataset:CIFAR, training... loss:0.70935
DPSH[12/150][10:39:42] bit:24, dataset:CIFAR, training... loss:0.70966
DPSH[13/150][10:39:42] bit:24, dataset:CIFA

100%|██████████| 500/500 [00:01<00:00, 437.19it/s]


Validation mAP: 0.5367
Testing combination: {'eta': 0.01, 'learning_rate': 1e-06, 'weight_decay': 0.0001}
DPSH[ 1/150][10:40:08] bit:24, dataset:CIFAR, training... loss:0.74204
DPSH[ 2/150][10:40:09] bit:24, dataset:CIFAR, training... loss:0.71709
DPSH[ 3/150][10:40:09] bit:24, dataset:CIFAR, training... loss:0.72391
DPSH[ 4/150][10:40:09] bit:24, dataset:CIFAR, training... loss:0.71573
DPSH[ 5/150][10:40:09] bit:24, dataset:CIFAR, training... loss:0.71713
DPSH[ 6/150][10:40:09] bit:24, dataset:CIFAR, training... loss:0.71432
DPSH[ 7/150][10:40:10] bit:24, dataset:CIFAR, training... loss:0.71324
DPSH[ 8/150][10:40:10] bit:24, dataset:CIFAR, training... loss:0.71294
DPSH[ 9/150][10:40:10] bit:24, dataset:CIFAR, training... loss:0.71059
DPSH[10/150][10:40:10] bit:24, dataset:CIFAR, training... loss:0.71107
DPSH[11/150][10:40:10] bit:24, dataset:CIFAR, training... loss:0.70961
DPSH[12/150][10:40:10] bit:24, dataset:CIFAR, training... loss:0.71073
DPSH[13/150][10:40:11] bit:24, dataset:CIF

...

Validation mAP: 0.5159
Testing combination: {'eta': 0.01, 'learning_rate': 1e-05, 'weight_decay': 1e-06}
DPSH[ 1/150][10:40:38] bit:24, dataset:CIFAR, training... loss:0.73307
DPSH[ 2/150][10:40:38] bit:24, dataset:CIFAR, training... loss:0.76372
DPSH[ 3/150][10:40:38] bit:24, dataset:CIFAR, training... loss:0.75678
DPSH[ 4/150][10:40:39] bit:24, dataset:CIFAR, training... loss:0.76779
DPSH[ 5/150][10:40:39] bit:24, dataset:CIFAR, training... loss:0.75720
DPSH[ 6/150][10:40:39] bit:24, dataset:CIFAR, training... loss:0.74789
DPSH[ 7/150][10:40:39] bit:24, dataset:CIFAR, training... loss:0.75822
DPSH[ 8/150][10:40:39] bit:24, dataset:CIFAR, training... loss:0.73627
DPSH[ 9/150][10:40:40] bit:24, dataset:CIFAR, training... loss:0.73116
DPSH[10/150][10:40:40] bit:24, dataset:CIFAR, training... loss:0.72762
DPSH[11/150][10:40:40] bit:24, dataset:CIFAR, training... loss:0.71367
DPSH[12/150][10:40:40] bit:24, dataset:CIFAR, training... loss:0.70962
DPSH[13/150][10:40:40] bit:24, dataset:CIFA

100%|██████████| 500/500 [00:01<00:00, 435.77it/s]


Validation mAP: 0.6807
Testing combination: {'eta': 0.01, 'learning_rate': 1e-05, 'weight_decay': 1e-05}
DPSH[ 1/150][10:41:09] bit:24, dataset:CIFAR, training... loss:0.72940
DPSH[ 2/150][10:41:09] bit:24, dataset:CIFAR, training... loss:0.75028
DPSH[ 3/150][10:41:09] bit:24, dataset:CIFAR, training... loss:0.76930
DPSH[ 4/150][10:41:09] bit:24, dataset:CIFAR, training... loss:0.77383
DPSH[ 5/150][10:41:10] bit:24, dataset:CIFAR, training... loss:0.74705
DPSH[ 6/150][10:41:10] bit:24, dataset:CIFAR, training... loss:0.75567
DPSH[ 7/150][10:41:10] bit:24, dataset:CIFAR, training... loss:0.74269
DPSH[ 8/150][10:41:10] bit:24, dataset:CIFAR, training... loss:0.72964
DPSH[ 9/150][10:41:10] bit:24, dataset:CIFAR, training... loss:0.72384
DPSH[10/150][10:41:11] bit:24, dataset:CIFAR, training... loss:0.72123
DPSH[11/150][10:41:11] bit:24, dataset:CIFAR, training... loss:0.71317
DPSH[12/150][10:41:11] bit:24, dataset:CIFAR, training... loss:0.71528
DPSH[13/150][10:41:11] bit:24, dataset:CIFA

100%|██████████| 500/500 [00:01<00:00, 428.50it/s]


Validation mAP: 0.6818
Testing combination: {'eta': 0.01, 'learning_rate': 1e-05, 'weight_decay': 0.0001}
DPSH[ 1/150][10:41:40] bit:24, dataset:CIFAR, training... loss:0.72661
DPSH[ 2/150][10:41:40] bit:24, dataset:CIFAR, training... loss:0.75627
DPSH[ 3/150][10:41:40] bit:24, dataset:CIFAR, training... loss:0.76741
DPSH[ 4/150][10:41:40] bit:24, dataset:CIFAR, training... loss:0.76695
DPSH[ 5/150][10:41:41] bit:24, dataset:CIFAR, training... loss:0.75335
DPSH[ 6/150][10:41:41] bit:24, dataset:CIFAR, training... loss:0.75525
DPSH[ 7/150][10:41:41] bit:24, dataset:CIFAR, training... loss:0.73644
DPSH[ 8/150][10:41:41] bit:24, dataset:CIFAR, training... loss:0.73446
DPSH[ 9/150][10:41:41] bit:24, dataset:CIFAR, training... loss:0.71660
DPSH[10/150][10:41:42] bit:24, dataset:CIFAR, training... loss:0.71310
DPSH[11/150][10:41:42] bit:24, dataset:CIFAR, training... loss:0.71283
DPSH[12/150][10:41:42] bit:24, dataset:CIFAR, training... loss:0.70569
DPSH[13/150][10:41:42] bit:24, dataset:CIF

In [ ]:
DPSH48Cifar = {'eta': 0.05, 'learning_rate': 1e-05, 'weight_decay': 1e-05}
DPSH32Cifar = {'eta': 0.1, 'learning_rate': 1e-05, 'weight_decay': 1e-05}
DPSH24Cifar = 
